In [ ]:

# import libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import plotly.express as px
from pywt import wavedec
from pywt import waverec
from scipy.signal import hilbert
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from pywt import *
import sys



In [ ]:
sys.path.insert(0, "D:\Arai4_Projects\spinwise_project\spinewise_amanuel\ML_Project\preprocessing")

In [ ]:
# import modules
from filter import apply_hilbert_filter
from filter import apply_haar_filter
from pca import apply_pca
from pca import determine_n_clusters
from segmentation import apply_segemntation
from sax import apply_symbolic_aggregation
from viz import viz_data
from viz import viz_output
from viz import viz_bar
from viz import viz_score
from get_data import load_file
from model import kmeans
from model import db_scan
from Single_File_SAX import single_df_SAX  






In [ ]:
l = [*range(2,10)]
l

In [ ]:
len(l)

In [ ]:
def start_cluster(file_type, file_name, features, preprocesses, model, number_cluster, y):
    """ initiate clustering """
    
    print("++++++++++ clustering initiated ++++++++++++++")
    print(f"file type : {file_type}")
    print(f"file name: {file_name}")
    print(f"features selected: {features}")
    print(f"Preprocessors selected: {preprocesses}")
    print(f"model selected: {model}")


    data = load_file(file_name, file_type)
    print("+++++++++++++++ data loaded +++++++++++++++++")
    print(f"size of data: {data.shape}")
    features_copy = features.copy()
    data_copy = data.copy()

    features_copy.append('ts_n')
    data = data[features_copy]
    viz_data(data,features, "Selected features vs time")

    global cluster

    for  preprocess in preprocesses:

        if preprocess == "filter":
            data, features = apply_hilbert_filter(data,features)
            print("+++++++++ hilbert_filters applied +++++++++++++++")
            viz_data(data, features,'Output of Hilbert Filter')

            data, features = apply_haar_filter(data, features)
            print("+++++++++ haar_filters applied +++++++++++++++")
            viz_data(data, features,'Output of Haar Filter')

        elif preprocess == "sax":
            data, features = single_df_SAX(data, features)
            print("+++++++++ symbolic aggregation applied ++++++++")
            viz_output(data,"SAX_vector","Output of SAX", y)

        elif preprocess == "segementation":
            continue

        elif preprocess == "pca":
            data, features = apply_pca(data,features, 2)
            print("++++++++++++++ pca applied ++++++++++++++++++++")
            viz_data(data, features,'Output of PCA')

        else:
            continue

    data_k = determine_n_clusters(data,features)
    print(data_k.head())
    viz_score(data_k, "Silhouette Score vs k ")

    if model == "k_means":
        cluster = kmeans(data[features], number_cluster)
        print("+++++++++++++ kmeans cluster trained ++++++++++")

    elif model == "db_scan":
        cluster = db_scan(data[features], number_cluster)
        print("++++++++++++ db_scan cluster trained ++++++++++")

    data['cluster'] = cluster.labels_
    return data, data_copy


In [ ]:
def user_input(file_type, file_name,features, preprocesses, model, number_cluster, y):
    """take input from a user
    """  

    data,data_copy = start_cluster(file_type, file_name, features, preprocesses, model, number_cluster,y)
    viz_output(data,'cluster','Output of Clusterng', y)
    
    if file_type == "labeled":
        viz_output(data_copy,'label','Labeled Clusters',y)
        data['label'] = data_copy['label']
        viz_bar(data, 'Clusters vs Labels')


    return data



In [ ]:
# user input 1
file_type = 'labeled' 
file_name = 'AW01-HBS-COMBINED-2022-11-8-14-16-48-1667918184-v1.4.3+lucina-buzz.csv'
features = ['acc_x_n','acc_y_n','acc_z_n']
y = 'acc_x_n'
preprocesses = ['filter','sax'] # write in order
model = 'db_scan'
number_cluster = 5
data= user_input(file_type, file_name,features, preprocesses, model, number_cluster,y)
data.head()


In [ ]:
# user input 2
file_type = 'labeled' 
file_name = 'AW01-HBS-COMBINED-2022-11-8-14-16-48-1667918184-v1.4.3+lucina-buzz.csv'
features = ['gyr_x_n','gyr_y_n','gyr_z_n','gyr_x_r','gyr_y_r','gyr_z_r']
y = 'gyr_x_n'
preprocesses = ['filter','pca','sax'] # write in order
model = 'k_means'
number_cluster = 5
data= user_input(file_type, file_name,features, preprocesses, model, number_cluster,y)
data.head()

In [ ]:
# user input_3
file_type = 'unlabeled' 
file_name = '063-COMBINED-2022-11-12-8-58-33-1668280845-v1.4.3+lucina-buzz.csv'
features = ['acc_x_n','acc_y_n','acc_z_n']
y ='acc_x_n'
preprocesses = ['filter','sax'] # write in order
model = 'k_means'
number_cluster = 5
data= user_input(file_type, file_name,features, preprocesses, model, number_cluster,y)
data.head()

In [ ]:
# user input_3
file_type = 'unlabeled' 
file_name = '063-COMBINED-2022-11-12-8-58-33-1668280845-v1.4.3+lucina-buzz.csv'
features = ['acc_x_n','acc_y_n','acc_z_n']
y = 'acc_x_n'
preprocesses = ['filter','pca','sax'] # write in order
model = 'k_means'
number_cluster = 5
data= user_input(file_type, file_name,features, preprocesses, model, number_cluster,y)
data.head()